# Notebook

In [1]:
import holidays

In [2]:
import pandas as pd
import holidays
import numpy as np
import yfinance as yf
from sklearn.model_selection import TimeSeriesSplit
from sklearn import linear_model
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet

In [3]:
#adquisition

In [4]:
data = yf.download('CSSPX.MI','2016-01-01','2023-06-01') # CSSPX.MI S&P500

[*********************100%***********************]  1 of 1 completed


In [5]:
data['HighLow_pct']= (data['High'] - data['Low']) / data['Close'] * 100
data['Pct_change'] = (data['Close'] - data['Open']) / data['Open'] * 100

In [6]:
data = data.reset_index()
data

,Date,Open,High,Low,Close,Adj Close,Volume,HighLow_pct,Pct_change
0,2016-01-04,179.119995,179.119995,170.570007,172.020004,172.020004,43555,4.970345,-3.963818
1,2016-01-05,173.889999,175.039993,172.750000,174.570007,174.570007,64667,1.311791,0.391056
2,2016-01-06,173.699997,173.699997,171.520004,172.800003,172.800003,17424,1.261570,-0.518131
3,2016-01-07,169.259995,169.320007,166.210007,168.669998,168.669998,51230,1.843837,-0.348574
4,2016-01-08,167.710007,168.339996,164.809998,164.809998,164.809998,53279,2.141860,-1.729181
...,...,...,...,...,...,...,...,...,...
1881,2023-05-25,401.399994,403.559998,400.739990,401.290009,401.290009,7277,0.702735,-0.027400
1882,2023-05-26,402.640015,408.399994,401.269989,407.570007,407.570007,12137,1.749394,1.224417
1883,2023-05-29,409.440002,409.829987,409.119995,409.809998,409.809998,4923,0.173249,0.090366
1884,2023-05-30,410.359985,411.339996,407.920013,408.549988,408.549988,7026,0.837103,-0.441076


In [7]:
# Time series scikit learn

tss = TimeSeriesSplit(n_splits = 3)

tss

X = data.drop(labels=['Adj Close', 'Open', 'High', 'Close', 'Low'], axis=1)
y = data['Adj Close']

data[:1500]['Date']

X_test = data.loc[1500:,['Date']]
y_test = data.loc[1500:, ['Adj Close']]
X_train = data.loc[:1499,['Date']]
y_train = data.loc[:1499, ['Adj Close']]

for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

X_train

ridge_model = linear_model.PoissonRegressor()
ridge_model.fit(X_train, y_train)
# Infer
y_pred_ridge = ridge_model.predict(X_test)

y_pred_ridge

y_test

stop

rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)
y_pred_rf = rf_model.predict(X_test)

pd.DataFrame(y_pred_rf)

## How do I predict times not passed?

def predict(ticker,days):
    yfin = yf.Ticker(ticker)
    print("Stock: ", yfin.info['name'])
    hist = yfin.history(period="max")
    hist = hist[['Adj Close']]
    hist.reset_index()
    hist = hist.rename({'Date': 'ds', 'Adj Close': 'y'}, axis='columns')
    print("Curent Data")
    print(hist.tail())
    m = Prophet(daily_seasonality=True)
    m.fit(hist)
    future = m.make_future_dataframe(periods=days)
    forecast = m.predict(future)
    print("Predicted Data")
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
    figure1 = m.plot(forecast)
    figure2 = m.plot_components(forecast)

predict("MSFT", 50)

In [8]:
data = data.reset_index()
data

,index,Date,Open,High,Low,Close,Adj Close,Volume,HighLow_pct,Pct_change
0,0,2016-01-04,179.119995,179.119995,170.570007,172.020004,172.020004,43555,4.970345,-3.963818
1,1,2016-01-05,173.889999,175.039993,172.750000,174.570007,174.570007,64667,1.311791,0.391056
2,2,2016-01-06,173.699997,173.699997,171.520004,172.800003,172.800003,17424,1.261570,-0.518131
3,3,2016-01-07,169.259995,169.320007,166.210007,168.669998,168.669998,51230,1.843837,-0.348574
4,4,2016-01-08,167.710007,168.339996,164.809998,164.809998,164.809998,53279,2.141860,-1.729181
...,...,...,...,...,...,...,...,...,...,...
1881,1881,2023-05-25,401.399994,403.559998,400.739990,401.290009,401.290009,7277,0.702735,-0.027400
1882,1882,2023-05-26,402.640015,408.399994,401.269989,407.570007,407.570007,12137,1.749394,1.224417
1883,1883,2023-05-29,409.440002,409.829987,409.119995,409.809998,409.809998,4923,0.173249,0.090366
1884,1884,2023-05-30,410.359985,411.339996,407.920013,408.549988,408.549988,7026,0.837103,-0.441076


In [9]:
data = data.rename({'Date': 'ds','Adj Close': 'y'}, axis='columns')
data = data[['ds', 'y']]

In [10]:
data

,ds,y
0,2016-01-04,172.020004
1,2016-01-05,174.570007
2,2016-01-06,172.800003
3,2016-01-07,168.669998
4,2016-01-08,164.809998
...,...,...
1881,2023-05-25,401.290009
1882,2023-05-26,407.570007
1883,2023-05-29,409.809998
1884,2023-05-30,408.549988


In [11]:
m = Prophet()
m.fit(data)

09:34:42 - cmdstanpy - INFO - Chain [1] start processing
09:34:42 - cmdstanpy - INFO - Chain [1] done processing
09:34:42 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
09:34:42 - cmdstanpy - INFO - Chain [1] start processing
09:34:42 - cmdstanpy - INFO - Chain [1] done processing
09:34:42 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization! Command 'C:\Users\marym\miniconda3\envs\tf\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=9450 data file=C:\Users\marym\AppData\Local\Temp\tmph3yc817i\pmdnzrng.json init=C:\Users\marym\AppData\Local\Temp\tmph3yc817i\9if2a2ee.json output file=C:\Users\marym\AppData\Local\Temp\tmph3yc817i\prophet_modele644e132\prophet_model-20230701093442.csv method=optimize algorithm=newton iter=10000' failed: 

In [ ]:
future = m.make_future_dataframe(periods=365)

future

In [ ]:
forecast = m.predict(future)
forecast

df2['date'] = pd.date_range(df2['date'].max()+pd.Timedelta(1,unit='d'),periods=365)


# Streamlit